In [ ]:
# Display an image in Jupyter notebook
from IPython.display import Image
from requests import get
from random import randint
# recreate your pokemon class here
#attributes needed... sprite, abilities, type, weight, height


class Pokemon:
    #Init function. It sets the majority of values to none, or empty sets, and then initializes the 'api_call' afterwards, filling the information out.
    def __init__(self, name):
        self.abilities = []
        self.name = name.lower()
        self.weight = None
        self.height = None
        self.types = []
        self.sprite = None
        self.species_link = None
        self.evo_stage = None
        self.poke_api_call()
        self.can_evolve = None
        
    #The poke_api_call. This is where the magic happens, and where we get the values filled out. If it fails, it returns an error message.
    def poke_api_call(self):
        response = get(f'https://pokeapi.co/api/v2/pokemon/{self.name}')
        
        if response.status_code == 200:
            data = response.json()
            
            ability_names = [item["ability"]["name"] for item in data["abilities"]]
            self.abilities = ability_names
            self.weight = data["weight"]
            self.height = data["height"]
            self.types = [item["type"]["name"] for item in data["types"]]
            self.sprite= data["sprites"]["versions"]["generation-v"]["black-white"]["animated"]["front_shiny"]
            if not self.sprite:
                self.sprite = data["sprites"]["front_default"]
            self.species_link = data["species"]["url"]
            self.evo_stage = self.evolve_stage(self.species_link)
            
            
            
        else:
            match response.status_code:
                case 404:
                    return "Error: 404 Not Found: The requested resource could not be found on the server."
                case 403: 
                    return "403 Forbidden: The server understood the request, but the client does not have permission to access the requested resource."
                case 400:
                    return "Error 400: The API Request was invalid"
    
    #The wonderful repr function, which converts our pokemon to text form. 
    def __repr__(self):
        return( f'Type: {self.types}, Abilities: {self.abilities} Evo Stage: {self.evo_stage}')
    
    #This function shows the pokemon's sprite
    def show_pokemon(self):
        display(Image(self.sprite, width= 200))

    #This recursive function is part of my attempts to solve the 'evolution method' problem. I figured that having the evo-stage would come in handy,
    #and I'm surprised that it's not available by default. 
    def evolve_stage(self, species_link):
        evolve_data = get(species_link).json()
        if not evolve_data["evolves_from_species"]:
            return 1
        else:
            return self.evolve_stage(evolve_data["evolves_from_species"]["url"])+1

    def evolve(self):
         pointer = get(get(self.species_link).json()["evolution_chain"]["url"]).json()["chain"]
         evolution = ""
         can_evolve = True
         while pointer["species"]["name"] != self.name:
             pointer = pointer["evolves_to"]
         match len(pointer["evolves_to"]):
            case 0:
                print("Error, pokemon cannot evolve")
                can_evolve = False
            case 1:
                 evolution = pointer["evolves_to"][0]["species"]["name"]
            case _: 
                 ev_list = [x["species"]["name"] for x in pointer["evolves_to"]]
                 index = [str(x) for x in range(len(ev_list))]
                 preview = ""
                 for i in range(len(index)):
                    preview = preview + index[i]+ ": " + ev_list[i].title() + " "
                 print(preview)
                 while evolution not in ev_list and evolution not in index:
                     evolution = input("Please enter either the pokemon name, or the number displayed.").lower()
                     if evolution in index:
                         evolution = index[evolution]
         if can_evolve:      
             self.show_pokemon()
             print(self.name.title() + " is evolving!")
             former = self.name
             self.name = evolution
            
             self.poke_api_call()
             self.show_pokemon()
             print("Congratulations," + former.title() + " has evolved into " + self.name.title())
        
    def random_pokemon(self):
        ids = str(randint(1,1281))
        self.name = get(f'https://pokeapi.co/api/v2/pokemon/{ids}').json()["name"]
        self.poke_api_call()


class Move_Tutor(Pokemon):
    def __init__(self, name, *args):
        Pokemon.__init__(self, name)
        self.move_list = {}
        for i in range(min(len(args), 4 )):
            self.move_list.update({i:args[i]})
    
    def show_moves(self):
        print("".join([str(i) + ": " + self.move_list[i] + "   " for i in self.move_list.keys()]))

    def show_all_moves(self):
         get_list = get(f'https://pokeapi.co/api/v2/pokemon/{self.name}').json()["moves"]
         list1 = {item["move"]["name"] for item in get_list}
    def teach_move(self): 
         get_list = get(f'https://pokeapi.co/api/v2/pokemon/{self.name}').json()["moves"]
         list1 = {item["move"]["name"] for item in get_list}
         print(list1)
         choice = ""
         while choice.lower() not in list1:
             choice = input("Please enter the move that you would like to learn.")
         keys = self.move_list.keys()
         
         if len(self.move_list) <4:
             for x in range(4):
                if x not in keys:
                    self.move_list.update({x: choice})
                    break
         else: 
             print(self.name.title() + " knows too many moves. Please pick one to replace")
             self.show_moves()
             choice2 = ""
             while choice2 not in ["0","1","2","3"]:
                 choice2 = input("Enter the number of the technique to replace")
             self.move_list.update({int(choice2): choice})
         self.show_moves()

             
         
psyduck = Move_Tutor("Snorlax")
psyduck.teach_move()
psyduck.teach_move()
psyduck.teach_move()
psyduck.teach_move()
psyduck.teach_move()
psyduck.show_moves()